In [54]:
import os
import re
from datetime import datetime
import json
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import worldnewsapi
from worldnewsapi.rest import ApiException
from bson import ObjectId

from functions.misc import NewsAPI_fetch, extract_author_names, extract_website_name, extract_categories

In [71]:
import torch
# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)

# If CUDA is available, print GPU name
if cuda_available:
    print("CUDA version:", torch.version.cuda)
    print("Number of GPUs:", torch.cuda.device_count())
    print("GPU Name:", torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PyTorch version: 2.0.1+cu118
CUDA available: True
CUDA version: 11.8
Number of GPUs: 1
GPU Name: NVIDIA GeForce GTX 1080 Ti


## Fetch new Records

In [49]:
import time
from uuid import uuid1
from dotenv import load_dotenv
from config.database import MongoDatabaseConnector
from pinecone import Pinecone
# to test asyn requests in upsert
# from pinecone.grpc import PineconeGRPC as Pinecone
# logging
from tqdm.auto import tqdm
from functions.logger import log_item_id, log_record_id_checker
# models
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from pinecone_text.sparse import BM25Encoder

load_dotenv()
MONGO_URI = os.getenv('MONGO_URI')
MONGO_DB_NAME = os.getenv('MONGO_DB_NAME')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

### Connections
## MongoDB
connection = MongoDatabaseConnector(MONGO_URI)
db = connection[MONGO_DB_NAME]
collection = db['articles']
## Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'newsrag'
index = pc.Index(index_name)
### Models
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("DeepMount00/Llama-3-8b-Ita")
# create the length function
def tokenizer_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=15,
    length_function=tokenizer_len,
    separators=["\n\n", "\n", " ", ""]
)
embedder = SentenceTransformer('nickprock/sentence-bert-base-italian-xxl-uncased')
sparse_embedder = BM25Encoder().default() # eventualmente caricare parametri addestrati


Connection enstablished


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\mattr\Documents\rag_project\Env2\NewsRAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Execution time: 21.10453224182129
Data uploaded successfully!


In [5]:
from functions.misc import NewsAPI_fetch
filtered_news_dicts = NewsAPI_fetch(max_results=2,
                                    NEWSAPI_API_KEY=NEWSAPI_API_KEY,
                                    min_date="2023-01-20",max_date="2023-01-20")

Retrieved 0 articles. Offset: 0/2. Total available: 0.
Retrieved 0 articles. Offset: 100/2. Total available: 0.
Retrieved 0 articles. Offset: 200/2. Total available: 0.
Retrieved 0 articles. Offset: 300/2. Total available: 0.
Retrieved 0 articles. Offset: 400/2. Total available: 0.
Retrieved 0 articles. Offset: 500/2. Total available: 0.
Retrieved 0 articles. Offset: 600/2. Total available: 0.
Retrieved 0 articles. Offset: 700/2. Total available: 0.
Retrieved 0 articles. Offset: 800/2. Total available: 0.
Retrieved 0 articles. Offset: 900/2. Total available: 0.
Retrieved 0 articles. Offset: 1000/2. Total available: 0.
Retrieved 0 articles. Offset: 1100/2. Total available: 0.
Retrieved 0 articles. Offset: 1200/2. Total available: 0.
Retrieved 0 articles. Offset: 1300/2. Total available: 0.
Retrieved 0 articles. Offset: 1400/2. Total available: 0.
Retrieved 0 articles. Offset: 1500/2. Total available: 0.
Retrieved 0 articles. Offset: 1600/2. Total available: 0.
Retrieved 0 articles. Offs

## Elaborazione JSON ricevuto

In [8]:
df = pd.DataFrame(filtered_news_dicts)
# Apply the function to the 'url' column and store the result in the 'category' column
df['category'] = df['url'].apply(lambda x: extract_categories(x))
# Testata
df['publisher'] = df['url'].apply(lambda x: extract_website_name(x))
# Autori
df['authors'] = df['authors'].apply(lambda x: extract_author_names(x))

In [47]:
data = df.to_dict(orient="records")

## update MongoDB

In [ ]:
### DATA STORAGE
# Insert data into the collection
start_time = time.time()
collection.insert_many(data) # if csv: data.to_dict(orient="records")
## delete all records : collection.delete_many(filter={})
end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time}")
print("Data uploaded successfully!")


## update vectorDB

In [72]:
def upsert_vectorDB(collection, index, text_splitter, embedder, sparse_embedder, batch_article_limit=3, log_processed_path='logs/ids_updated_vecDB.log'):
    """# Define the batch limit (number of articles to process in each batch)
    batch_article_limit = 3
    index=index
    log_processed_path="logs/ids_processed.log"
    """
    # Retrieve processed IDs from logs
    processed_ids = log_record_id_checker(log_processed_path)
    object_ids = [ObjectId(oid) for oid in list(set(processed_ids))]
    query = {'_id': {'$nin': object_ids}}
    projection = {'_id': 1}

    # Main loop for processing articles in batches
    acab = np.arange(0, 1+len(list(collection.find(query, projection))) // batch_article_limit)
    for _ in acab:
        processed_ids = log_record_id_checker(log_processed_path)
        object_ids = [ObjectId(oid) for oid in list(set(processed_ids))]
        query = {'_id': {'$nin': object_ids}}
        results = collection.find(query).limit(batch_article_limit)
        data = list(results)

        if not data:
            break

        texts = []
        metadatas = []
        for record in tqdm(data):
            metadata = {
                'doc_id': str(record['_id']),
            }
            record_texts = text_splitter.split_text(record['text'])
            record_metadatas = [{
                "chunk_no": j, "context": text, **metadata
            } for j, text in enumerate(record_texts)]
            texts.extend(record_texts)
            metadatas.extend(record_metadatas)

        if texts:
            ids = [str(uuid1()) for _ in range(len(texts))]
            embeds = embedder.encode(texts, device=device)#'cuda'
            ##############sparse_embeds = generate_sparse_vectors(texts)
            sparse_embeds = sparse_embedder.encode_documents(texts)
            # print(ids) # uncomment to debug
            # print(sparse_embeds) # uncomment to debug
            ##########################vectors = list(zip(ids, sparse_embeds, embeds, metadatas))+
            vectors = []
            for _id, sparse, dense, metadataa in zip(ids, sparse_embeds, embeds, metadatas):
                ### MODIFICARE SPARSE QUI
                # floats = [float(x) for x in list(sparse.values())]
                # sparse = dict({'indices': list(sparse.keys()), 'values': floats})
                # print(sparse)
                vectors.append({
                'id': _id,
                'sparse_values': sparse,
                'values': dense,
                'metadata': metadataa
                })
            # print(vectors) # uncomment to debug
            # Store the processed data in Pinecone
            index.upsert(vectors)# , async_req=True)# pero mi pare di capire che gli va dato un vettore alla volta allora andrebbe nel for di sopra (e quindi anche senza l'.append)

            for vec in vectors:
                item_id = str(vec['metadata']["doc_id"])
                chunk_no = str(vec['metadata']["chunk_no"])
                log_item_id(item_id=item_id, chunk_no=chunk_no, chunk_id=str(vec['id'])
                            , log_file_path=log_processed_path)

        print(f"Total number of articles processed in this batch: {len(data)}")

    print(f"Total number of articles processed: {len(acab) * batch_article_limit -(batch_article_limit-1)}")
    return print("Update complete")  

In [73]:
upsert_vectorDB(collection=collection, index=index,
                text_splitter=text_splitter, embedder=embedder, sparse_embedder=sparse_embedder,
                batch_article_limit=100, 
                log_processed_path='logs/ids_updated_vecDB.log')

Batches: 100%|██████████| 9/9 [00:02<00:00,  3.76it/s]


Total number of articles processed in this batch: 100


Batches: 100%|██████████| 8/8 [00:01<00:00,  4.74it/s]


Total number of articles processed in this batch: 100


Batches: 100%|██████████| 9/9 [00:02<00:00,  3.70it/s]


...

Total number of articles processed in this batch: 100


Batches: 100%|██████████| 8/8 [00:01<00:00,  5.22it/s]


Total number of articles processed in this batch: 100


Batches: 100%|██████████| 1/1 [00:00<00:00, 86.89it/s]


Total number of articles processed in this batch: 1
Total number of articles processed: 26201
Update complete


In [68]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1}},
 'total_vector_count': 1}

# Remove records

In [37]:
to_delete_items = ["665f1222756e8549ca815604", "665f1222756e8549ca815601"]
# mongodb Delete operation
Obj_tbd = [ObjectId(x) for x in to_delete_items]
query = {'_id' : Obj_tbd}
collection.remove(query)

In [ ]:
# Pinecone Delete synchronization
import json
filepath='logs/ids_updated_vectorDB.log'
# parse del log per ottenere gli id dei vettori e rimuoviamo dal log
# Initialize an empty list to store record_ID values
record_ids_tbr = []
# Open the file for reading
with open(filepath, 'r') as fr:
    lines = fr.readlines()
    with open(filepath, 'w') as fw:
        # Iterate over each line in the file
        for line in lines:
            if ("INFO:root" in line):
                # Find the position of the JSON string within the log line
                start_pos = line.find("{")
                end_pos = line.rfind("}") + 1
                json_str = line[start_pos:end_pos]
                # Parse the JSON string to a dictionary
                log_data = json.loads(json_str)
                if log_data['record_ID'] not in to_delete_items:
                    fw.write(line)
                else:
                    record_ids_tbr.append(log_data['chunk_id'])
    fw.close()

index.delete(ids=record_ids_tbr)

In [56]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}